In [1]:
import pandas as pd
import plotly.graph_objs as go
import dash
from dash import dcc, html
import dash_cytoscape as cyto

In [86]:
sheet_id = "1DERvsSuI838s8ZRNG69sP4Pz2tgnHvoTvGljUk0ldQc"  # your Google Sheet ID
People = "0"
Marriages = "1981493742"  # the gid of the 'Marriages' sheet

url = "https://docs.google.com/spreadsheets/d/{}/export?format=csv&gid={}"

marriages_df = pd.read_csv(url.format(sheet_id,Marriages))
people_df = pd.read_csv(url.format(sheet_id,People))

name_to_id = {f"{row['First']} {row['Last']}": row['P_id'] for _, row in people_df.iterrows()}


In [87]:
def assign_xpos(people_df, marriages_df, spacing_x=1, spacing_y=1):
    """
    Assign xpos based on generation and relationship structure.
    Returns an updated people_df with xpos.
    """
    from collections import defaultdict

    # Group people by generation
    gen_to_people = defaultdict(list)
    for _, row in people_df.iterrows():
        gen_to_people[row['Generation']].append(row['P_id'])

    # Keep track of assigned x positions
    xpos_map = {}
    current_x = 0

    # Start from the oldest generation and go down
    sorted_gens = sorted(gen_to_people.keys())

    for gen in sorted_gens:
        people_in_gen = gen_to_people[gen]

        # Handle couples together
        already_placed = set()
        for _, marriage in marriages_df.iterrows():
            a = marriage['PersonA']
            b = marriage['PersonB']

            # Make sure they are both in this generation
            if a in people_in_gen and b in people_in_gen and a not in already_placed and b not in already_placed:
                xpos_map[a] = current_x
                xpos_map[b] = current_x + spacing_x // 2
                current_x += spacing_x * 2  # extra spacing between couples
                already_placed.update([a, b])

        # Place individuals not in marriages or already placed
        for pid in people_in_gen:
            if pid not in xpos_map:
                xpos_map[pid] = current_x
                current_x += spacing_x

    # Assign x positions to children centered under their parents
    for _, row in marriages_df.iterrows():
        a = row['PersonA']
        b = row['PersonB']
        children = [c.strip() for c in str(row['Children']).split(",") if c.strip()]

        # Use midpoint of the parents
        if a in xpos_map and b in xpos_map:
            parent_mid_x = (xpos_map[a] + xpos_map[b]) / 2

            # Spread children around the midpoint
            num_children = len(children)
            child_spacing = spacing_x
            start_x = parent_mid_x - (child_spacing * (num_children - 1)) / 2

            for i, child in enumerate(children):
                if child not in xpos_map:
                    xpos_map[child] = start_x + i * child_spacing

    # Update the DataFrame with xpos
    people_df['xpos'] = people_df['P_id'].map(xpos_map)

    return people_df
people_df = assign_xpos(people_df, marriages_df)


In [41]:
import networkx as nx

G = nx.DiGraph()

# Add nodes and edges (marriages and children)
G.add_node("P1")
G.add_node("P2")
G.add_edge("P1", "M1")  # P1 part of marriage
G.add_edge("P2", "M1")
G.add_edge("M1", "C1")  # Marriage has child

# Use Graphviz to compute layout
pos = nx.nx_agraph.graphviz_layout(G, prog='dot')  # or use 'neato'/'twopi'

# Use pos to set node positions in Cytoscape


ImportError: requires pygraphviz http://pygraphviz.github.io/

In [88]:
# Make node lists with json type object
nodes = []
for _, row in people_df.iterrows():
    nodes.append({'data': {'id': row['P_id'], 'label': f"{row['First']} {row['Last']}"},
                  'position': {'x': row['xpos']*100, 'y': row['Generation']*100}  
    })

# Make edge lists with json type object
edges = []
# Create marriage edges and child edges
for _, row in marriages_df.iterrows():
    person_a_id = name_to_id[row['PersonA']]
    person_b_id = name_to_id[row['PersonB']]

    marriage_x = (people_df.loc[people_df['P_id'] == person_a_id, 'xpos'].values[0]*100 + 
                  people_df.loc[people_df['P_id'] == person_b_id, 'xpos'].values[0]*100) / 2

    nodes.append({'data': {'id': row['M_id'],},
                  'position': {'x': marriage_x, 'y': row['Generation']*100},
                  'style': {'width': '10%', 'height': '10%'}  
    })
    
    # Create edges for the marriage between PersonA and PersonB
    edges.append({'data': {'source': person_a_id, 'target': row['M_id'], 'label': 'Marriage'},
                  'style': {'line-color': 'blue'}
    })
    edges.append({'data': {'source': person_b_id, 'target': row['M_id'], 'label': 'Marriage'},
                  'style': {'line-color': 'blue'}
    })
    
    # Create child edges from the middle of the marriage line
    # Average x position of the two parents to place the child edge
    marriage_x = (people_df.loc[people_df['P_id'] == person_a_id, 'xpos'].values[0]*100 + 
                  people_df.loc[people_df['P_id'] == person_b_id, 'xpos'].values[0]*100) / 2
    if type(row['Children']) != float:
       
        for child in row['Children'].split(";"):
            child_name = child.strip()
            child_id = name_to_id[child_name] if child_name in name_to_id else None
            if child_id:
                edges.append({
                    'data': {'source': row['M_id'], 'target': child_id, 'label': 'Child'},
                    'style': {'line-color': 'green'},
                })


In [89]:
# Create Dash App
app = dash.Dash(__name__)

# Cytoscape layout options
layout = {
    'name': 'preset',  # Use preset positions
    'animate': True
}

# Dash Cytoscape component
app.layout = html.Div([
    cyto.Cytoscape(
        id='family-tree',
        elements=nodes + edges,
        layout=layout,
        style={'width': '100%', 'height': '600px'}
    )
])

# Run the app
if __name__ == '__main__':
    app.run(debug=True)